# Calculating DCJ distance between two unsigned, circular and/or linear genomes with no duplicates

Credit:
- https://github.com/mlliou112/py-dcj (python version)
- ChatGPT for converting to Julia 

In [1]:
# import statements
using Base.Iterators
using Parameters
using DataStructures

abstract type AbstractGene end

In [2]:
# telomere

@with_kw struct Telomere <: AbstractGene
    reverse::Bool=false  # TODO
end

function Base.isequal(a::Telomere, b)
    return typeof(a) == typeof(b)
end

function Base.show(t::Telomere)
    return "."
end

In [3]:
# gene

@with_kw struct Gene <: AbstractGene
    id::Int
    dna::Char
    reverse::Bool=false
end

function Base.isequal(a::Gene, b::Gene)
    return typeof(a) == typeof(b) && a.id == b.id
end

function Base.show(m::Gene)
    if m.reverse
        print("-" * string(m.id))
    else
        print(string(m.id))
    end
end

In [4]:
# chromosome

mutable struct Chromosome
    genes::Vector{AbstractGene}
end

function Chromosome(genes::String, id_counter::Ref{Int64}, id_to_char::Dict{Int, Char}, char_to_id::Dict{Char, Int64}, target::Bool)
    content = AbstractGene[]

    if !((genes[1] == '.') == (genes[end] == '.'))
        throw(ArgumentError("Linear geneosome must start and end with telomeres."))
    end

    for (i, gene) in pairs(genes)
        dna = only(lowercase(string(gene)))
        rev = isuppercase(gene)
        telomere = gene == '.'        
        if telomere 
            push!(content, Telomere())
        else  # gene 
            id = id_counter[]

            if target  
                id_to_char[id] = gene
                if gene in keys(char_to_id)
                    push!(char_to_id[dna], id)
                else 
                    char_to_id[dna] = id
                end 
                id_counter[] += 1

            else  # source str 
                ## assuming no duplicates
                id = char_to_id[dna]
            end
            push!(content, Gene(id, dna, rev))
        end 
    end    
    
    return Chromosome(content)
end

Chromosome

In [5]:
# Genome

mutable struct Genome
    data::Vector{Chromosome}
    id_to_char::Dict{Int, Char}
    char_to_id::Dict{Char, Int}
end

In [6]:
# funcs convert bt genome & str 
function genome_to_string(A)
end


function string_to_genome(s, id_counter, id_to_char, char_to_id, target)
     """
    Converts string to type Genome 
    """

    chrom_list_str = Vector{String}()

    chrom = ""
    new_chrom = true 
    linear = false

    for (i, g) in pairs(s)
        # if creating a new chromosome 
        if new_chrom 
            if chrom == ""
                linear = g == '.'
            else  # chrom isn't empty 
                linear = chrom[1] == '.'
            end 

            chrom = chrom * g
            new_chrom = false  
            if i == length(s) # last element is circular chrom 
                push!(chrom_list_str, chrom)
               
            end 
        
        # appending more genes to chromosome 
        else  
            if linear  
                chrom = chrom * g 
                if g == '.' || i == length(s)  # find another telomere or end of str
                    push!(chrom_list_str, chrom) 
                    chrom = ""
                    new_chrom = true 
                end 
            else  # circular
                if g =='.'  # close circular chrom (won't incude telomere in chrom)
                    push!(chrom_list_str, chrom)
                    chrom = "."  # put telomere in next chrom 
                    new_chrom = true 
                elseif i == length(s)  # reach end of str 
                    chrom = chrom * g
                    push!(chrom_list_str, chrom)
                    chrom = "."
                    new_chrom = true 
                else 
                    chrom = chrom * g
                end 
            end 
        end 
    end

    # convert chrom_list_str (list of strings) to chrom_list
    chrom_list = Chromosome[]
    
    for c_str in chrom_list_str
        c = Chromosome(c_str, id_counter, id_to_char, char_to_id, target)
        push!(chrom_list, c)
    end 
    
    return Genome(chrom_list, id_to_char, char_to_id)
end 

string_to_genome (generic function with 1 method)

In [7]:
# gene end (used in adj)

@with_kw struct GeneEnd
    gene::AbstractGene
    head::Bool=true
end

# adjacency
mutable struct Adjacency 
    left::GeneEnd
    right::GeneEnd
end

function Base.isequal(a::GeneEnd, b::GeneEnd)
    return a.id == b.id && a.head == b.head
end

function Base.show(ge::GeneEnd)
    if ge.head
        print(string(ge.gene.dna) *":h")
    else
        print(string(ge.gene.dna) * ":t")
    end

end

function Base.isequal(a::Adjacency, b::Adjacency)
     # order of the gene ends doesn't matter
    if a.left == b.left && a.right == b.right || a.left == b.right && a.right == b.left  
        return true 
    end 
    return false 
end 

function Base.show(adj::Adjacency)
    left = ""
    right = ""
    
    if adj.left.gene == Telomere() 
        left = "."
    else 
        left_gene = adj.left
        (left_gene.head == true) ? left = left_gene.gene.dna * ":h" : left = left_gene.gene.dna * ":t" 
    end
    
    if adj.right.gene == Telomere() 
        right = "."
    else 
        right_gene = adj.right
        (right_gene.head == true) ? right = right_gene.gene.dna * ":h" : right = right_gene.gene.dna * ":t"
    end 

    print("(", left, ",", right, ")")
end 

function Base.show(adj::Adjacency, add_spacer::Bool)
    left = ""
    right = ""
    
    if adj.left.gene == Telomere() 
        left = "."
    else 
        left_gene = adj.left
        (left_gene.head == true) ? left = left_gene.gene.dna * ":h" : left = left_gene.gene.dna * ":t" 
    end
    
    if adj.right.gene == Telomere() 
        right = "."
    else 
        right_gene = adj.right
        (right_gene.head == true) ? right = right_gene.gene.dna * ":h" : right = right_gene.gene.dna * ":t"
    end 

    if add_spacer 
        print("(", left, "---", right, ")")
    else
        print("(", left, ",", right, ")")
    end 
end 


function Base.show(adj_list::Vector{Adjacency})
    for adj in adj_list 
        show(adj) 
    end 
end 

function Base.show(adj_list::Vector{Adjacency}, idx1::Int, idx2::Int)
    r = min(idx1, idx2)
    s = max(idx1, idx2)
    for (i, adj) in pairs(adj_list)  
        show(adj, i == r || i == s) 
    end 
end 

function Base.show(adj_set::Set{Adjacency})
    for adj in adj_set 
        show(adj) 
    end 
end 

In [8]:
# genome to adj list and set 

function genome_to_adj_listset_helper(chrom::Chromosome, genes::Vector{AbstractGene}, adj_list::Vector{Adjacency})
    linear = genes[1] == Telomere()
    
    # handle first gene end 
    if linear 
        left_end_adj = GeneEnd(Telomere(), false)
    else  # circular 
        left_end_adj = GeneEnd(genes[1], !genes[1].reverse)
    end 

    # handle gene bt first and last 
    for gene in genes
        if gene == Telomere() || gene == genes[1] 
            continue 
        end 
        reversed = gene.reverse  
        
        right_gene_adj = GeneEnd(gene, reversed)  # if reversed, right_end_adj is head (& left_end_adj is tail) 
        adj = Adjacency(left_end_adj, right_gene_adj)

        push!(adj_list, adj)

        left_end_adj = GeneEnd(gene, !reversed)
    end 

    # handle last gene end 
    if linear 
        right_end_adj = GeneEnd(Telomere(), false)
    else   
        right_end_adj = GeneEnd(genes[1], genes[1].reverse)
    end

    return left_end_adj, right_end_adj

end 


function genome_to_adj_listset(genome::Genome)
    adj_list = Vector{Adjacency}()
    adj_set = Set{Adjacency}()

    for chrom in genome.data
        genes = chrom.genes
        reversed = genes[1].reverse  
        
        if length(genes) == 1  # single gene edge case   
            left_end_adj = GeneEnd(genes[1], reversed)  # if reversed, left_end_adj is head (& right_end_adj is tail) 
            right_end_adj = GeneEnd(genes[1], !reversed) 
        elseif length(genes) == 2  # two gene edge case 
            gene1 = genes[1]
            gene2 = genes[2]

            left_end_adj = GeneEnd(gene1, !gene1.reverse)
            right_end_adj = GeneEnd(gene2, gene2.reverse) 
            adj = Adjacency(left_end_adj, right_end_adj)
            push!(adj_list, adj)

            reversed = genes[2].reverse 
            left_end_adj = GeneEnd(gene2, !gene2.reverse)
            right_end_adj = GeneEnd(gene1, gene1.reverse) 
        else # > 2 gene in chrom  
            left_end_adj, right_end_adj = genome_to_adj_listset_helper(chrom, genes, adj_list)
        end 
        
        adj = Adjacency(left_end_adj, right_end_adj)
        push!(adj_list, adj)
        push!(adj_set, adj)
    end 
    return adj_list, adj_set
end 

genome_to_adj_listset (generic function with 1 method)

In [9]:
# process adj list

function process_adj_list_helper(ge::GeneEnd, idx::Ref{Int}, gid_to_loc::DefaultDict{Int, Vector{Int}})
    if ge.gene == Telomere() 
        return
    end  

    if ge.head == true
        gid_to_loc[ge.gene.id][2] = idx[]
    else 
        gid_to_loc[ge.gene.id][1] = idx[]
    end
end 

function process_adj_list(adj_list:: Vector{Adjacency})
    geneid_to_location = DefaultDict{Int, Vector{Int}}(() -> zeros(Int, 2)) # tail = idx 1, head = 2 in array
    idx = Ref{Int}(1)

    for adj in adj_list
        process_adj_list_helper(adj.left, idx, geneid_to_location)
        process_adj_list_helper(adj.right, idx, geneid_to_location)
        idx[] += 1
    end 
    
    return geneid_to_location 
end 

process_adj_list (generic function with 1 method)

In [10]:
# helpers for dcj operations and distance

# finds target gene end in source adjacency list 
# returns index and left/right in the source adj list
function find_tar_ge_in_src_adjlist(target_ge::GeneEnd, src_gid_to_l::DefaultDict{Int, Vector{Int}}, tar_gid_to_l::DefaultDict{Int, Vector{Int}}, src_adj_list::Vector{Adjacency})
    (target_ge.head == true) ? th_idx = 2 : th_idx = 1
    # (left_tar_ge == true) ? tar_lr = 'L' : tar_lr = 'R'
        
    gene_id = target_ge.gene.id

    src_ge_idx = src_gid_to_l[gene_id][th_idx]

    adj = src_adj_list[src_ge_idx]
    if adj.left.gene.id == gene_id
        src_ge = adj.left
        # src_lr = 'L'
    else 
        src_ge = adj.right 
        # src_lr = 'R' 
    end 

    # print("\n   GENEEND: ")
    # show(target_ge)
    # print("\n     src genome idx: ", src_ge_idx, " ", src_lr)
    # print("\n     target genome idx: ", tar_gid_to_l[gene_id][th_idx], " ", tar_lr, "\n")

    return [src_ge, src_ge_idx]
end 

function other_adjacency_end(ge::GeneEnd, adj::Adjacency)
    if adj.left == ge
        return adj.right
    else
        return adj.left
    end 
end

function combine_ge(u::GeneEnd, u_idx::Int, v::GeneEnd, v_idx::Int)
    if u_idx < v_idx 
        u_lt_v = true 
    else 
        u_lt_v = false 
    end 

    if !u_lt_v 
        return Adjacency(v, u)
    else 
        return Adjacency(u, v)
    end

end 

function reassign_locs(gid_to_l, adj_list, u_idx, v_idx)
    for (i, adj) in pairs(view(adj_list, min(u_idx, v_idx):max(u_idx, v_idx)))
        left = adj.left 
        right = adj.right 

        idx = Ref{Int}(i + min(u_idx, v_idx) - 1)

        process_adj_list_helper(left, idx, gid_to_l)
        process_adj_list_helper(right, idx, gid_to_l)
    end 
end 



reassign_locs (generic function with 1 method)

In [14]:
function update_adj_set(p::GeneEnd, q::GeneEnd, u::GeneEnd, v::GeneEnd, u_idx::Int, v_idx::Int, src_adj_list::Vector{Adjacency}, src_adj_set::Set{Adjacency}) 
    #  replace adj u and v in A by ( {p, q} and (u\{p}) U (v\{q}) )
    pq = combine_ge(p, u_idx, q, v_idx)

    other_ge_u = other_adjacency_end(u, src_adj_list[u_idx])
    other_ge_v = other_adjacency_end(v, src_adj_list[v_idx])
    excluding_pq = combine_ge(other_ge_u, u_idx, other_ge_v, v_idx) 
    
    #  remove adj with u, v in src_adj_set 
    delete!(src_adj_set, src_adj_list[u_idx])
    delete!(src_adj_set, src_adj_list[v_idx])
    
    #  add adjusted adjacencies to src_adj_set
    push!(src_adj_set, pq)
    push!(src_adj_set, excluding_pq)

    # print("  src_adj_set:")
    # show(src_adj_set)
    # print("\n")
end 

function update_adj_list_gid_loc_dict() 
    # update adj lists 
        print("\nDCJ Operation (Cycle) :: ")
        show(src_adj_list, u_idx, v_idx)
        
        print("\n")
        show(pq)
        show(excluding_pq)
        if u_idx < v_idx 
            src_adj_list[u_idx] = pq
            src_adj_list[v_idx] = excluding_pq
            reverse!(view(src_adj_list, u_idx+1:v_idx-1))
        else 
            src_adj_list[u_idx] = excluding_pq
            src_adj_list[v_idx] = pq
            reverse!(view(src_adj_list, v_idx+1:u_idx-1))
        end  
    
        # update gid-->loc dicts 
        reassign_locs(src_gid_to_l, src_adj_list, u_idx, v_idx)
end 

# find dcj operations and distance

function find_dcj_dist_ops(src_adj_list::Vector{Adjacency}, tar_adj_list::Vector{Adjacency}, src_gid_to_l::DefaultDict{Int, Vector{Int}}, tar_gid_to_l::DefaultDict{Int, Vector{Int}}, src_adj_set::Set{Adjacency}, target_adj_set::Set{Adjacency})
    count = 0  
    # cycles = 0 
    # odd_paths = 0 
    telomere_idxs = Vector{Int}

    print("indexing through target adj list...")
    
    # for each adj {p, q} in target genome 
    for (i, adj) in pairs(tar_adj_list) 
        print("\n\n")
        show(src_adj_list)
         
        print("\nADJ ", i, " :::: ")
        show(adj)

        p = adj.left  
        q = adj.right  

        if p.gene == Telomere() || q.gene == Telomere()  # telomeres handled in next loop
            push!(telomere_idxs, i)  
        end 

        #  let u be element of genome A that contains p 
        #  let v be element of genome A that contains q 
        u, u_idx = find_tar_ge_in_src_adjlist(p, src_gid_to_l, tar_gid_to_l, src_adj_list)  
        v, v_idx = find_tar_ge_in_src_adjlist(q, src_gid_to_l, tar_gid_to_l, src_adj_list)  
        
        #  if u != v, replace u and v in A by {p, q} and (u\{p}) U (v\{q})
        if u_idx != v_idx             
            update_adj_set(p, q, u, v, u_idx, v_idx, src_adj_list, src_adj_set)
            # update_adj_list_gid_loc_dict()
            
            count += 1 
        end
        # cycles += 1  # TODO cycle counting     
    end 

    # for each telomere {p} in genome B 
    for idx in telomere_idxs
        telo_adj = tar_adj_list[idx]
        telo_ge = other_adjacency_end(telo_adj, Telomere())

        #  let u be element of genome A that contains p 
        u, u_idx = find_tar_ge_in_src_adjlist(p, src_gid_to_l, tar_gid_to_l, src_adj_list)  

        #  if u is an adjacency, then replace u in A by {p} and (u\{p})
        if other_adjacency_end(tar_adj_list[u_idx], u) != Telomere() 
            # update_adj_set(p, u, u_idx,src_adj_list, src_adj_set)
            # update_adj_list_gid_loc_dict()

            count += 1 
        end 

        #  odd paths++  # TODO odd path counting 
    end 

    return count 
    # return cycles, odd_paths
end 



src = "abc"
target = "cba"

calculate_distance(src, target)


SRC ADJ LIST
(a:h,b:t)(b:h,c:t)(c:h,a:t)
|
v
TARGET ADJ LIST
(c:h,b:t)(b:h,a:t)(a:h,c:t)

*************
indexing through target adj list...

(a:h,b:t)(b:h,c:t)(c:h,a:t)
ADJ 1 :::: (c:h,b:t)

(a:h,b:t)(b:h,c:t)(c:h,a:t)
ADJ 2 :::: (b:h,a:t)

(a:h,b:t)(b:h,c:t)(c:h,a:t)
ADJ 3 :::: (a:h,c:t)

3

In [12]:
# adjacency graph 

mutable struct AdjacencyGraph
    cycles::Int
    odd_paths::Int
end

function Base.show(dict::DefaultDict{Int, Vector{Int}}, id_to_char::Dict{Int, Char})
    for (key, value) in dict
        # println("$(key.dna) (id:$(key.id)) => $(value)")
        println(id_to_char[key], " $(key) => $(value)")
    end
end 

function AdjacencyGraph(src::Genome, target::Genome)

    # process genomes (not explicitly creating the adj graph)    
    src_adj_list, src_adj_set = genome_to_adj_listset(src)        
    target_adj_list, target_adj_set = genome_to_adj_listset(target)        
    
    print("SRC ADJ LIST\n")
    show(src_adj_list)
    print("\n|\nv\nTARGET ADJ LIST\n")
    show(target_adj_list)
    print("\n\n*************\n")
    
    src_geneid_to_location = process_adj_list(src_adj_list)   
    target_geneid_to_location = process_adj_list(target_adj_list)
    
    if src.id_to_char != target.id_to_char
        throw(ArgumentError("ERROR: Source ID to char dict is not equal to target's."))
    end 

    # print("SRC GENE --> LOCATION DICT\n")
    # show(src_geneid_to_location, src.id_to_char)  
    # print("\n\n", "TARGET GENE --> LOCATION DICT\n")
    # show(target_geneid_to_location, src.id_to_char)
    # print("\n*************\n\n")

    # use adj graph representation to find dcj distance and operations 
    # cycles, odd_paths = 
    find_dcj_dist_ops(src_adj_list, target_adj_list, src_geneid_to_location, target_geneid_to_location, src_adj_set, target_adj_set) 
    
    # return AdjacencyGraph(cycles, odd_paths)
end 

"""
assumptions 
genomes A and B have the same letters of the same multiplicity; telomeres don't matter 
no empty chromosomes (2 telomeres ..)
"""

function calculate_distance(src::String, target::String)
    # check_conditions(src, target)

    id_counter = Ref{Int}(1)
    id_to_char = Dict{Int, Char}()
    char_to_id = Dict{Char, Int}()

    target_genome = string_to_genome(target, id_counter, id_to_char, char_to_id, true)
    src_genome = string_to_genome(src, id_counter, id_to_char, char_to_id, false)

    ag = AdjacencyGraph(src_genome, target_genome)

    # dcj_distance = src_genome.num_genes - ag.cycles - (ag.ab_paths / 2)  # or target_genome.num_genes

    # return dcj_distance
end

calculate_distance (generic function with 1 method)